In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv(r'C:\Users\98all\Documents\GitHub\ECE196_SP21\Hate-2001-2019-0.csv')

In [4]:
#Fill empty values with None for Weapon Type and clean up typos in dataset
df['WeaponType'].fillna('None',inplace = True)
df['WeaponType'].replace('Other ( bottle, rocks, spitting)', 'Other (bottle, rocks, spitting)', inplace=True)
df['WeaponType'].replace('Firearm (unknown whether handgun, rifle or shotgun', 'Firearm (unknown whether handgun, rifle or shotgun)', inplace=True)

#Replace rest of NaN values with 'Unknown'
df_new = df.replace(np.nan, 'Unknown', regex=True)

#rearrange columns to be more easily readable
columns_titles = ['MostSeriousBias','MostSeriousBiasType','TotalNumberOfVictims','MostSeriousVictimType','TotalNumberOfSuspects','SuspectsRaceAsAGroup','MostSeriousUcr','MostSeriousUcrType','WeaponType','Offensive_Act','MostSeriousLocation','County','NCIC','ClosedYear','MonthOccurrence']
df_new=df_new.reindex(columns=columns_titles)

#One hot encode all necessary columns
df_new = pd.concat([df_new,pd.get_dummies(df_new['MostSeriousBias'], prefix='Bias')],axis=1)
df_new.drop(['MostSeriousBias'],axis=1, inplace=True)
TotalNumberOfVictims = df_new['TotalNumberOfVictims']
df_new.drop(['TotalNumberOfVictims'],axis=1, inplace=True)
df_new = pd.concat([df_new,TotalNumberOfVictims],axis=1)
df_new = pd.concat([df_new,pd.get_dummies(df_new['MostSeriousVictimType'], prefix='VictimType')],axis=1)
df_new.drop(['MostSeriousVictimType'],axis=1, inplace=True)
TotalNumberOfSuspects = df_new['TotalNumberOfSuspects']
df_new.drop(['TotalNumberOfSuspects'],axis=1, inplace=True)
df_new = pd.concat([df_new,TotalNumberOfSuspects],axis=1)
df_new = pd.concat([df_new,pd.get_dummies(df_new['SuspectsRaceAsAGroup'], prefix='SuspectsRace')],axis=1)
df_new.drop(['SuspectsRaceAsAGroup'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['MostSeriousUcr'], prefix='Ucr')],axis=1)
df_new.drop(['MostSeriousUcr'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['MostSeriousUcrType'], prefix='UcrType')],axis=1)
df_new.drop(['MostSeriousUcrType'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['WeaponType'], prefix='Weapon')],axis=1)
df_new.drop(['WeaponType'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['Offensive_Act'], prefix='Offense')],axis=1)
df_new.drop(['Offensive_Act'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['MostSeriousLocation'], prefix='Location')],axis=1)
df_new.drop(['MostSeriousLocation'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['County'], prefix='County')],axis=1)
df_new.drop(['County'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['NCIC'], prefix='Agency')],axis=1)
df_new.drop(['NCIC'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['ClosedYear'], prefix='Year')],axis=1)
df_new.drop(['ClosedYear'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['MonthOccurrence'], prefix='Month')],axis=1)
df_new.drop(['MonthOccurrence'],axis=1, inplace=True)

In [5]:
#separate data into labels and data points
y_labels = df_new['MostSeriousBiasType']
x_data = df_new.iloc[:, 1:778]

In [6]:
#split data into training and testing sets and shuffle before splitting
x_Train, x_Test, y_Train, y_Test = train_test_split(x_data, y_labels, shuffle = True, test_size = .2)

In [7]:
#scale the data/normalize the data so that the features do not broadly vary
scaler = StandardScaler()
scaler.fit(x_Train)

x_Train = scaler.transform(x_Train)
x_Test = scaler.transform(x_Test)

In [8]:
#build the Knn classifier
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=2)
classifier.fit(x_Train, y_Train)

KNeighborsClassifier(n_neighbors=2)

In [9]:
y_pred = classifier.predict(x_Test)

In [36]:
#print out statistics to show how well our classifier did
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_Test, y_pred))

                         precision    recall  f1-score   support

             Disability       1.00      0.83      0.91        12
                 Gender       1.00      0.73      0.84        11
   Gender Nonconforming       0.93      0.74      0.83        77
Race/Ethnicity/Ancestry       0.74      0.95      0.83      2773
               Religion       0.77      0.53      0.63       768
     Sexual Orientation       0.84      0.38      0.52       981

               accuracy                           0.76      4622
              macro avg       0.88      0.69      0.76      4622
           weighted avg       0.77      0.76      0.73      4622



In [11]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state = 0).fit(x_Train, y_Train)


In [21]:
y_prediction = clf.predict(x_Test)

In [39]:
print(classification_report(y_Test, y_prediction))

                         precision    recall  f1-score   support

             Disability       1.00      1.00      1.00        12
                 Gender       1.00      1.00      1.00        11
   Gender Nonconforming       1.00      1.00      1.00        77
Race/Ethnicity/Ancestry       1.00      1.00      1.00      2773
               Religion       1.00      1.00      1.00       768
     Sexual Orientation       1.00      1.00      1.00       981

               accuracy                           1.00      4622
              macro avg       1.00      1.00      1.00      4622
           weighted avg       1.00      1.00      1.00      4622



In [26]:
clf.predict_proba(x_Test)

array([[1.66011568e-06, 1.11946112e-06, 8.19098784e-06, 9.99880700e-01,
        4.39135995e-05, 6.44158329e-05],
       [1.39874301e-06, 1.01202335e-06, 4.85208200e-06, 9.99894686e-01,
        2.74085964e-05, 7.06425237e-05],
       [1.10741090e-06, 8.33731849e-07, 2.45137688e-06, 1.63378916e-04,
        9.99814540e-01, 1.76887374e-05],
       ...,
       [2.73626406e-06, 1.90242033e-06, 7.82323327e-06, 2.55007644e-04,
        9.13389700e-05, 9.99641191e-01],
       [3.13523475e-06, 2.13682148e-06, 2.11028541e-05, 9.99731420e-01,
        7.47888563e-05, 1.67415958e-04],
       [1.61133791e-06, 9.38750815e-07, 9.00116958e-06, 9.99902391e-01,
        2.93613723e-05, 5.66966252e-05]])

In [27]:
clf.predict_log_proba(x_Test)

array([[-1.33086233e+01, -1.37026631e+01, -1.17124761e+01,
        -1.19307114e-04, -1.00332865e+01, -9.65015110e+00],
       [-1.34799366e+01, -1.38035589e+01, -1.22361027e+01,
        -1.05319514e-04, -1.05046539e+01, -9.55787828e+00],
       [-1.37134858e+01, -1.39973540e+01, -1.29188607e+01,
        -8.71943842e+00, -1.85477373e-04, -1.09425824e+01],
       ...,
       [-1.28089170e+01, -1.31723836e+01, -1.17584126e+01,
        -8.27421703e+00, -9.30093303e+00, -3.58872919e-04],
       [-1.26728065e+01, -1.30561911e+01, -1.07661023e+01,
        -2.68615799e-04, -9.50084166e+00, -8.69502908e+00],
       [-1.33384457e+01, -1.38787158e+01, -1.16181560e+01,
        -9.76140199e-05, -1.04358306e+01, -9.77779587e+00]])

In [28]:
clf.score(x_data,y_labels)

0.6848437635246256

In [30]:
print(clf.coef_)

[[-1.63990802e-03 -4.75416380e-03 -7.32411657e-03 ... -7.59764689e-03
  -2.16774980e-03  1.31126838e-02]
 [-1.02792405e-03 -3.10860940e-03 -4.82509971e-03 ...  1.22106015e-03
  -5.52963790e-04  7.76926292e-03]
 [-5.38577352e-03 -1.72739002e-02 -2.70887901e-02 ...  1.32249032e-02
   4.29676819e-03 -4.78122313e-03]
 [ 1.61242992e-01  4.28958700e-01  6.81172623e-01 ... -8.51666104e-03
   1.28183773e-02 -1.77914882e-03]
 [-7.26616713e-02 -1.74602026e-01 -2.84607343e-01 ... -6.29000566e-03
  -2.82530243e-02 -7.11714743e-03]
 [-8.05277151e-02 -2.29220000e-01 -3.57327274e-01 ...  7.95835020e-03
   1.38585925e-02 -7.20442729e-03]]


In [31]:
clf.densify()

LogisticRegression(random_state=0)

In [32]:
coef = clf.coef_

In [34]:
print(coef.shape)

(6, 777)


In [37]:
confusion_matrix(y_Test, y_prediction)

array([[  12,    0,    0,    0,    0,    0],
       [   0,   11,    0,    0,    0,    0],
       [   0,    0,   77,    0,    0,    0],
       [   0,    0,    0, 2773,    0,    0],
       [   0,    0,    0,    0,  768,    0],
       [   0,    0,    0,    0,    0,  981]], dtype=int64)